# CD++: Puertos Dinámicos

En este notebook veremos específicamente como creamos puertos dinámicos con la herramienta CD++. Lo haremos anlizando nuestro caso de uso, con puertos de salida (aunque reproducible para puertos de entrada). 

En nuestro modelo contabamos con un Dispatcher que se encargaba de enviar jobs a N servers. Una posibilidad frente a esto sería, crear como veníamos haciendo N puertos de salida en el dispatcher declarando explícitamente cada uno. La contra de esto es que, para modificar la cantidad de puertos se debía modificar el código y recompilarlo. Luego el mismo no sería reutilizable con distintos .ma que tengan distinta cantidad de servers.

A continuación analizamos la alternativa.

## Puertos dinámicos: utilizando un mapa de puertos

La otra solución que utilizamos fue declarar un mapa de puertos en el dispatcher.h sin ningún tamaño inicial. De esta manera, mapearemos el servidor i con el puerto i. (0 <= i < n). Aclaración: utilizamos un mapa porque necesitabamos asociar al servidor i con el puerto i porque además tiempo guardabamos el estado de los mismos. Pero si esto no fuera necesario en nuestro modelo, se podría utilizar únicamente un vector de puertos y utilizarlos libremente.

Veamos como queda:

In [9]:
! head -n 60 src/dispatcher.h | tail -n 2

    // This is a map of server_id => port
    map<int, Port*> jobsToProcess;


Luego, en el .ma definimos los puertos, la cantidad de servers y el estado inicial de cada uno:

In [10]:
! cat models/dispatcher.ma 

[top]
components : dispatcher@dispatcher server0@server server1@server server2@server server3@server server4@server 
in : newJob jobDone serverStackInfo
out : requestJob server0 server1 server2 server3 server4
link : newJob newJob@dispatcher
link : jobDone jobDone@dispatcher 
link : serverStackInfo serverStackInfo@dispatcher
link : requestJob@dispatcher requestJob
link : server0@dispatcher job@server0
link : server1@dispatcher job@server1
link : server2@dispatcher job@server2
link : server3@dispatcher job@server3
link : server4@dispatcher job@server4

[dispatcher]
in : newJob jobDone serverStackInfo
out : requestJob server0 server1 server2 server3 server4
numberOfServers : 5

server0: free
server1: free
server2: off
server3: free
server4: free


Una vez declarados los mismos en el .ma, debemos proceder a leer estos parámetros desde el código de c++. En el archivo del dispatcher, dispatcher.cpp, lo que haremos es:
   1. Leer el parámetro _numberOfServers_
   2. Luego iterar sobre esta cantidad, y en la i-ésima iteración leer el parámetro compuesto por la palabra server y el número de la iteración:  _"server" + string(i)_.
   3. Agregamos el outputPort al modelo y guardamos un puntero hacia el mismo: **_Port* linkToServer = &addOutputPort(portName)_**
   4. Agregamos el puntero al puerto asociado al server i-ésimo en en el mapa: **_jobsToProcess[i] = linkToServer_**
   5. Luego al usar el puerto para enviar un mensaje, debemos obtenerlo de nuestro mapa y derreferenciarlo, por ejemplo: **_sendOutput(msg.time(), *jobsToProcess[i], this->jobID)_**
   
A continuación veamos el código utilizado para los puntos de 1 a 4:

```
Dispatcher::Dispatcher(const string &name) : 
    Atomic(name),
    newJob(addInputPort("newJob")),
    jobDone(addInputPort("jobDone")),
    serverStatus(addInputPort("serverStatus")),	
    requestJob(addOutputPort("requestJob")),
    jobID(Real(0)),
    requestedJob(false),
    jobArrived(false),
    serverToDispatch(-1)
{
    numberOfServers = stoi( ParallelMainSimulator::Instance().getParameter( description(), "numberOfServers" ) );

    for (register int i = 0; i < numberOfServers; i++){
        string serverStateVarName = "server" + to_string(i);
        // now we initialize server status
        if( ParallelMainSimulator::Instance().existsParameter( description(), serverStateVarName ) ){
            string serverState( ParallelMainSimulator::Instance().getParameter( description(), serverStateVarName ) ) ;
            statusOfServers[i] = serverState;
        } else {
            statusOfServers[i] = DEFAULT_SERVER_STATE;
        }

        // now we create output ports to servers
        string portName = "server" + to_string(i);
        if (DISPATCHER_DEBUGGING_ENABLED){
            cout << "[DISPATCHER::Dispatcher] Creating port for: " << portName << endl;
        }
        Port* linkToServer = &addOutputPort(portName);
        jobsToProcess[i] = linkToServer;
    }

    if (statusOfServers.size() > numberOfServers){
        MTHROW(MException("Error! Hay más servers que los permitidos"))
    }
}
```

### Integrar puertos dinámicos con Pringles

Como sabemos, al utilizar la herramienta pringles, debemos declarar los puertos en la metadata que lee de los archivos headers, lo que hace necesario que la información de los puertos este escrita en los mismos. 

Sin embargo, la herramienta también permite agregar puertos dinámicamente a los modelos una vez descubiertos (es decir, después de la lectura de los headers). Esto se puede realizar utilizando los métodos: **_add_outport(portName)_**, **_add_inport(portName)_**

Veamos un ejemplo en el que instanciamos los servidores y el dispatcher:

```
from pringles.models.errors import PortNotFoundException

dispatcher = Dispatcher('dispatcher', **dispatcherDefaultConfig)
servers = {}
for i in range(number_of_servers):
    server_name = 'server' + str(i)
    servers[i] = Server(server_name, **serverOnDefaultConifg)
    try:
        dispatcher.get_port(server_name)
    except PortNotFoundException:
        dispatcher.add_outport(server_name)
```